In [11]:
#文件中数字、字母、空格等出现次数统计

def count_file(fname):
    digits = letters = spaces = others = 0
    infile = open(fname, encoding="utf_8")
    for line in infile:
        for c in line:
            if c.isdigit():
                digits += 1
            elif c.isalpha():
                letters += 1
            elif c.isspace():
                spaces += 1
            else:
                others += 1
    infile.close()
    print("文件中数字共：\t", digits, "digits;")
    print("文件中字母共：\t", letters, "letters;")
    print("文件中空格共：\t", spaces, "blank chars;")
    print("文件中其他字符共：\t", others, "other chars.")
count_file("data/data1.txt")

('\xe6\x96\x87\xe4\xbb\xb6\xe4\xb8\xad\xe6\x95\xb0\xe5\xad\x97\xe5\x85\xb1\xef\xbc\x9a\t', 15286, 'digits;')
('\xe6\x96\x87\xe4\xbb\xb6\xe4\xb8\xad\xe5\xad\x97\xe6\xaf\x8d\xe5\x85\xb1\xef\xbc\x9a\t', 139308, 'letters;')
('\xe6\x96\x87\xe4\xbb\xb6\xe4\xb8\xad\xe7\xa9\xba\xe6\xa0\xbc\xe5\x85\xb1\xef\xbc\x9a\t', 43372, 'blank chars;')
('\xe6\x96\x87\xe4\xbb\xb6\xe4\xb8\xad\xe5\x85\xb6\xe4\xbb\x96\xe5\xad\x97\xe7\xac\xa6\xe5\x85\xb1\xef\xbc\x9a\t', 10003, 'other chars.')


In [24]:
#计算文件中数据的平均值和方差 
#1、打开文件，读一遍数据求出平均值，关闭文件；重新打开文件，读一遍数据，求出方差




#2、利用文件的当前位置重定位功能，第一遍读完，重新设定到文件开头，再读一遍







#3、在计算平均值的过程中，用一个表记录所有数据；在做方差计算时扫描该数据表






#假设文件中的数据如下形式：
#2 3 5.9
#3.8 4 9.0
#90 3
#采用上面提到的第三种方法实现，定义函数mean_v

def mean_v(fname):
    num=0
    fsum=0.0
    data=[]#第三种方法中的需要的记录数据的表
    infile=open(fname)
    for line in infile:
        for s in line.split():#每行的数据切分
            x=float(s)  #转换为浮点数
            data.append(x)#追加到data的后面
            fsum+=x  #计算和
            num+=1  #统计个数
            
    if num == 0:
        return (0, 0, 0)
    mean = fsum / num  #计算平均值

    fsum = 0
    for x in data:#重新扫描数据表，计算方差
        fsum += (x - mean)**2
    return (num, mean, (fsum / num)**0.5)

#调用函数
mean_v("aa.txt")

(8, 19.7875, 26.374487554263496)

In [36]:
###----读取文件中的数据  示例------###



#注意：文件中是空格隔开的数字，转换为float浮点数




##第1种解决方案

#提取由空白字符分隔的文件中的数值数据   
def read_floats(fname):
    infile=open(fname)#打开文件的open函数，建立文件对象
    #read函数得到文件中所有内容的字符串（--与文件大小一样的字符串--），对该字符串调用split得到一个表
    #（--构建一个表，还要构造切分得到的一组字符串--），其中的元素是表示数值的一系列字符串；
    #然后调用map把float应用于这些字符串（--map逐个创建一组浮点数--）；得到一个生成浮点数的迭代器（map函数的结果是一个迭代器）；
    #最后用list做出这些浮点数的表（--含有所有浮点数的表[]--）
    
    #如果被读文件很大，文件字符串和几个表的体量也很大，还要建立并同时保存大量小字符串（每个串对应一个数）。即使程序需要维持这些数据，
    #并反复使用这些数据，read建立的大字符串也没必要。
    
    #为避免创建无价值的大型数据对象，可以考虑上面例子的 mean_v（）函数的做法
    numbers=list(map(float,infile.read().split())) #文件中是空格隔开的数字，转换为float浮点数
    
    infile.close()
    return numbers
read_floats("data/aa.txt")

[2.0, 4.0, 6.0]

In [46]:
##改进第一种解决方案

#改进read_floats（）函数         
#解决思路：每次调用得到文件里的一个浮点数，尽量避免建立大型数据对象
#定义两个函数：open_float（）函数打开文件；next_float（）函数逐一读出文件里的浮点数。next_float采用“缓冲区”式工作方式：用一个缓冲区
#（一个表）保存文件中的一行信息，每次调用的时候根据表里的信息生成一个浮点数返回。一旦表中内容用完，然后再从文件中读取一行。分解该行的内容
#后存入缓冲区





#两个函数协作完成任务，并且next_float使用open_float建立的文件对象，即：文件对象的传递问题。
#函数next_float使用的缓冲区在函数调用之后
#还应该继续存在，，以便下次调用时继续使用。因此，需要生命周期长的变量记录这些数据对象。next_float和open_float两个函数是相互独立的，
#因此只能用全局变量



#定义需要的全局变量
infile=None
nlist=[]#保存浮点数字符串
crt=0
#定义打开文件的函数
def open_float(fname):
    global infile
    infile=open(fname)
   
#定义读取一行数据的函数
def next_float():
    global nlist,crt #缓冲式处理，使用全局变量
    if crt==len(nlist):#当前用完一行，再读一行。其中变量crt类似一个指针
        line=infile.readline()  #读一行数据
        if not line:  #空字符串表示文件已经处理完毕
            infile.close()  #关闭文件
            return None
        nlist=line.split()  #切分单词
        crt=0  #记录当前项的下标
    x=nlist[crt]
    crt+=1
    #print("指针位置：",crt)
    return float(nlist[crt-1])

# 调用方式:
# 首先调用 open_floats(filename) 初始化相关功能
# 然后每次调用 next_float() 得到下一个浮点数

if __name__ == "__main__":
    ####调用打开文件的函数：
    open_float("data/data.dat")
    for i in range(50):#根据文件定义循环
        print(next_float())
    print("\n")

    

#分析上面的实现过程：函数中创建的最长字符串就是文件里最长的一行，通过切分产生的表只包括一行里的浮点数，不会产生很大的浮点数
#存在的问题也很明显：使用了几个全局变量，其中保存的是函数内部使用的数据，不太安全。
#针对存在的问题可以这样处理：把这两个函数以及相关的数据定义为一个模块，用import导入时把它们都留在模块对象里，并不导入主程序的全局名字空间
#但是全局变量还是会出现在模块的名字空间里


#### 缺点：用了几个全局变量，功能可能受到干扰


In [ ]:
#如何解决上面的问题？
#如何实现：在满足问题需求的同时又做到变量局部化

#定义一个函数，处理文件打开工作，为了局部化，文件打开中和打开后使用的变量应该定义为函数的局部变量
def radd_floats():
    nlist=[]
    infile=open(fname)
    crt=0
    #...
    #下面的内容该如何实现呢？这个函数应该返回什么呢？
    #应该是返回能获得的浮点数的功能，python语言功能性的对象就是函数，因此，应该返回一个能读取出文件数据的函数，使得能通过调用该函数逐一
    #得到文件中的浮点数   显然，radd_floats函数返回的函数应该是在该函数内部定义的，需要返回一个局部（内部）定义的函数-----闭包
   

#### 另一种实现，其中使用了局部函数和局部变量
#### 返回局部定义的函数对象（实际上带着局部环境）

def read_floats(fname):
    nlist = []
    infile = open(fname)
    crt = 0
    #定义局部函数
    def next_float():
        nonlocal nlist
        nonlocal crt
        if crt == len(nlist): # 一行已经用完
            line = infile.readline()
            if not line: # line是空串，整个文件已经处理完
                infile.close()
                return None
            nlist = line.split()
            crt = 0
        crt += 1
        return float(nlist[crt - 1])

    return next_float # 返回局部定义的函数对象

#程序开始
if __name__ == "__main__":
    nextf1 = read_floats("data/data.dat")

    for i in range(10):
        print(nextf1())
